In [1]:
#Inception Module


# Load pickled data
import pickle
import numpy as np
import tensorflow as tf 

# TODO: Fill this in based on where you saved the training and testing data

training_file = "train_gray_balanced_contrast.p"
testing_file = "test_final_gray.p"
validation_file = "validation_gray_balanced.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
# with open(testing_file, mode='rb') as f:
#     test = pickle.load(f)
with open(validation_file, mode='rb') as f:
    validation = pickle.load(f)
    
X_train_old, y_train = train['features'], train['labels']
# X_test, y_test = test['features'], test['labels']
X_validation_old, y_validation = validation['features'], validation['labels']


In [2]:
from sklearn.preprocessing import normalize

X_new = X_train_old#.astype(float)
X_new = X_new.astype(float)
X_new-=128.0
X_train = X_new /255.0

X_val = X_validation_old#.astype(float)
X_val = X_val.astype(float)
X_val-=128.0
X_validation = X_val /255.0

# X_test = X_test#.astype(float)
# X_test = X_test.astype(float)
# X_test-=128.0
# X_test = X_test /255.0

/opt/conda/lib/python3.5/site-packages/PIL/Image.py:85: RuntimeWarning: The _imaging extension was built for another  version of Pillow or PIL
  warnings.warn(str(v), RuntimeWarning)


In [3]:

from tensorflow.contrib.layers import flatten
import tensorflow as tf


def conv2d(x,weight,bias,strides=1,padding='SAME'):
    layer = tf.nn.conv2d(x,weight,[1,strides,strides,1],padding=padding)
    layer = tf.nn.bias_add(layer,bias)
    layer = tf.nn.relu(layer)
    return layer

def maxpool(x,k=2,padding='SAME',stride=2):
    kernel_size = [1,k,k,1]
    return tf.nn.max_pool(x,ksize=kernel_size,strides=[1,stride,stride,1],padding=padding)
    

In [4]:
# Based on the blog here
#https://chatbotslife.com/resnets-highwaynets-and-densenets-oh-my-9bb15918ee32#.jkp9iv6s0
import tensorflow.contrib.slim as slim
def resUnit(input_layer,depth=64,kernel_size=3):
    resnet_1 = slim.batch_norm(input_layer,activation_fn=None)
    resnet_1 = tf.nn.relu(resnet_1)
    resnet_1 = slim.conv2d(resnet_1,depth,[kernel_size,kernel_size],activation_fn=None)
    print(resnet_1.get_shape())

    resnet_2 = slim.batch_norm(resnet_1,activation_fn=None)
    resnet_2 = tf.nn.relu(resnet_2)
    resnet_2 = slim.conv2d(resnet_2,depth,[kernel_size,kernel_size],activation_fn=None)
    print(resnet_2.get_shape())
    resnet_tensor = input_layer + resnet_2
    print(resnet_tensor.get_shape())
    return resnet_tensor


def fully_connected(in_tensor,layers,keep_prob):
    mu = 0
    sigma = 0.1
    in_dim = np.product(in_tensor.get_shape().as_list()[1:])
    tensor_flatten = flatten(in_tensor)
    print(in_dim)
    for each in layers:
        weights = tf.Variable(tf.random_normal([in_dim, each],mu,sigma,dtype=tf.float32))
        bias =  tf.Variable(tf.zeros(each))

        fc1_drop = tf.nn.dropout(tensor_flatten, keep_prob)    
        fc2 = tf.add(
        tf.matmul(fc1_drop, weights),
        bias)
        # TODO: Activation.
        tensor_flatten = tf.nn.tanh(fc2)
        in_dim = each
        print(in_dim)
    
    return tensor_flatten
         
        
    


In [5]:
INPUT_DEPTH = 1
tf.reset_default_graph()
x = tf.placeholder(tf.float32, (None, 32, 32, INPUT_DEPTH))
y = tf.placeholder(tf.int32, (None))
keep_prob = tf.placeholder(tf.float32)
one_hot_y = tf.one_hot(y, 43)
layer1 = slim.conv2d(x,9,[3,3],normalizer_fn=slim.batch_norm)
resnet_tensor = resUnit(layer1,depth=9)
logits = fully_connected(resnet_tensor,[500,500,43],keep_prob)

(?, 32, 32, 9)
(?, 32, 32, 9)
(?, 32, 32, 9)
9216
500
500
43


In [6]:
rate = 0.0005
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
# Gradient clipping
gvs = optimizer.compute_gradients(loss_operation)
capped_gvs = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gvs]
optimizer.apply_gradients(capped_gvs)

training_operation = optimizer.minimize(loss_operation)

In [7]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y,keep_prob:1.0})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [8]:
EPOCHS = 100
BATCH_SIZE = 256

In [9]:
from sklearn.utils import shuffle
from imblearn.over_sampling import RandomOverSampler
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
model_file = "./resnet_v2"
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    num_examples =len(X_train)#392090
    from pathlib import Path
    

    my_file = Path(model_file)
    #if my_file.is_file():
    saver.restore(sess,model_file)
    print("Model restored...")
    print("Training...")
    print()
    ros = RandomOverSampler()
    x_index = np.arange(0,len(y_train),1)
    x_index_resampled, y_resampled = ros.fit_sample(np.expand_dims(x_index,axis=1), y_train)
    x_in = np.squeeze(x_index_resampled)
    X_resampled = X_train[x_in,:,:,]
    X_train, y_train = shuffle(X_resampled, y_resampled)
      
    for i in range(EPOCHS):
  
        X_train, y_train = shuffle(X_train, y_train)
        print(len(y_train)," # of training examples")
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y,keep_prob:0.9})
            
        validation_accuracy = evaluate(X_validation, y_validation)
        training_accuracy = evaluate(batch_x, batch_y)
        mean_cross_entropy = sess.run(loss_operation, feed_dict={x: batch_x, y: batch_y,keep_prob:1.0})
        print("EPOCH {} ...".format(i+1))
        print("Training Accuracy = {:.3f}".format(training_accuracy))
        print("Cross entropy = {:.4f}".format(mean_cross_entropy))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        if(i%10==0):
            saver.save(sess, model_file)
            print("Model saved")
        
    saver.save(sess, model_file)
    print("Model saved")

Instructions for updating:
Use `tf.global_variables_initializer` instead.


KeyboardInterrupt: 

In [ ]:
#!pip install imblearn